In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import datetime

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/ecommerce-sales-data-in-2019/2. ecommerce sales data 2019.csv')
df

In [ ]:
df.info()

In [ ]:
df.isna().any()

**Investigating the NaN values**

In [ ]:
df[df['Postal Code'].isna()]
#df[df['Order ID']=='Order ID']

In [ ]:
df.describe()

Order ID doesn't have unique values, so duplicates from Order ID can't be eliminated. Rather duplicates from whole table needs elimination.

In [ ]:
df[df.duplicated()]

**Cleaning the dataframe**

In [ ]:
#cleaning duplicates
df_clean=df.drop_duplicates().copy()
df_clean.duplicated().sum()

In [ ]:
#removing nulls
df_clean['Postal Code'].fillna('N/A',inplace=True)
df_clean.isna().any()

In [ ]:
#cleaning whiteapaces
def df_wide_strip(data):
    for col in data:
        data[col]=data[col].str.strip()
    return data

df_clean=df_wide_strip(df_clean)
list(df_clean['City'].unique())

In [ ]:
df_clean.info()
#all columns are object type

In [ ]:
df_clean[df_clean['Order ID']=='Order ID']

In [ ]:
def clean_junk(data):
    for column in data.columns:
        data.drop(data.loc[data[column]==column].index,inplace=True)
    return data

df_clean=clean_junk(df_clean)

**Changing data types of Order ID, Quantity Ordered, Price Each, Order Date**

In [ ]:
df_clean['Order ID']=pd.to_numeric(df_clean['Order ID'])
df_clean['Order Date']=pd.to_datetime(df_clean['Order Date'])
df_clean['Quantity Ordered']=pd.to_numeric(df_clean['Quantity Ordered'])
df_clean['Price Each']=pd.to_numeric(df_clean['Price Each'])
df_clean.info()

**Saving to new file**

In [ ]:
df_clean.to_excel(' ecommerce sales data 2019_cleaned.xlsx',index=False)

# Data Analysis

In [ ]:
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
df_eda=df_clean.copy()

In [ ]:
df_eda.head()

In [ ]:
df_eda['Order Month']=df_eda['Order Date'].dt.strftime('%B')
df_eda

In [ ]:
#Month Dictionary
months = ['January','February','March','April','May','June','July','August','September','October','November','December']


In [ ]:
df_eda['Revenue']=df_eda['Quantity Ordered']*df_eda['Price Each']
revenue_pm=df_eda.groupby('Order Month')[['Revenue']].sum().reindex(months)
revenue_pm

In [ ]:
plt.figure(figsize=(10,6))
plt.title('Sales by Month')
sns.lineplot(x=revenue_pm.index,y=revenue_pm['Revenue'])
plt.xticks(rotation=45)
plt.show()

**Let's see what brings in the most revenue**

In [ ]:
per_product=df_eda.groupby(['Product'])[['Product','Revenue','Quantity Ordered']].sum().sort_values(by='Revenue',ascending=False)
per_product